# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [27]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [30]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [31]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [32]:
try: 
    session.execute(""" 
        CREATE KEYSPACE IF NOT EXISTS sparkfy
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)
    

#### Set Keyspace

In [33]:
try:
    session.set_keyspace('sparkfy')
except Exception as e:
    print(e)

## First Query

### The first request from the database was to access the app history and retrive the artist, song title and duration given a session ID and the order of the song in that session. 

#### - session_id=338 and item_in_session = 4

#### For this query, the chosen PRIMARY KEYS are session_id and item_in_session, since in every session the user can only listen to one song at a time.  

In [34]:
create_table_app_history = "CREATE TABLE IF NOT EXISTS app_history"
create_table_app_history = create_table_app_history + \
        "(\
            session_id int, \
            item_in_session int, \
            artist_name text, \
            song_title text, \
            song_length float, \
            PRIMARY KEY(session_id, item_in_session) \
          )"

try: 
    session.execute(create_table_app_history)
except Exception as e:
    print(e)
                    

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        insert_app_history = "INSERT INTO app_history \
            (\
            session_id,\
            item_in_session, \
            artist_name, \
            song_title, \
            song_length \
            )"
        insert_app_history = insert_app_history + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(insert_app_history, \
                            ( \
                             int(line[8]), \
                             int(line[3]), \
                             line[0], \
                             line[9],\
                             float(line[5])\
                            ) \
                       )

In [36]:
query = "SELECT artist_name, song_title, song_length FROM app_history WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, ",", row.song_title,",", row.song_length) 

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


## Second Query

### The second request for the database was to try and verify the user song prefence by using as input the session id and user id. The songs (output) were listed based on the order they were played. 

#### session_id=182 and user_id=10
#### For this query, the PRIMARY KEY (session_id and user_id) were chosen, in this order, to decrease the number of instances that appeared during the first search. The item_in_session is a partition key that helps ordering the results. 

In [37]:
create_table_user_preference = "CREATE TABLE IF NOT EXISTS user_preference"
create_table_user_preference = create_table_user_preference + \
    "(\
     user_id int, \
     session_id int, \
     item_in_session int, \
     artist_name text, \
     song_title text,  \
     user_name text, \
     PRIMARY KEY((user_id,session_id),item_in_session) \
     )"

try: 
    session.execute(create_table_user_preference)
except Exception as e:
    print(e)
                    

In [38]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        insert_user_preference= "INSERT INTO user_preference \
            (\
            user_id, \
            session_id, \
            item_in_session, \
            artist_name, \
            song_title, \
            user_name \
            )"
        insert_user_preference = insert_user_preference + 'VALUES (%s,%s,%s,%s,%s,%s)'
        userName = line[1] + " " + line[4]
        session.execute(insert_user_preference, \
                        (\
                         int(line[10]),\
                         int(line[8]), \
                         int(line[3]), \
                         line[0],\
                         line[9], \
                         userName\
                        ) \
                       )

In [39]:
query1 = "SELECT artist_name, song_title, user_name FROM user_preference WHERE user_id=10 AND session_id=182 "

try:
    rows = session.execute(query1)
    
except Exception as e:
    print(e)

for row in rows:
    print (row.artist_name, ",",row.song_title, ",",row.user_name) 

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


## Third Query

### The third request was to collect all users who heard a specific song.
#### song_title = 'All Hands Against His Own'
#### For this query, the necessary PRIMARY KEY is the song title as partition key and user_id as the clustering key, since if only the song title was selected the query would display only one user.

In [40]:
create_table_song_history = "CREATE TABLE IF NOT EXISTS song_history"
create_table_song_history = create_table_song_history + \
        "(\
        song_title text,\
        user_id int, \
        first_name text, \
        last_name text, \
        PRIMARY KEY (song_title,user_id)\
        )"

try: 
    session.execute(create_table_song_history)
except Exception as e:
    print(e)


In [41]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        insert_song_history = "INSERT INTO song_history (song_title, user_id, first_name, last_name)"
        insert_song_history = insert_song_history + "VALUES (%s,%s,%s,%s)"
        session.execute(insert_song_history,(line[9],int(line[10]),line[1],line[4]))


In [42]:
query2= "SELECT first_name, last_name FROM song_history WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(query2)
    
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name) 

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [25]:
session.execute('DROP TABLE IF EXISTS app_history')
session.execute('DROP TABLE IF EXISTS user_preference')
session.execute('DROP TABLE IF EXISTS song_history')


### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()